In [51]:
import boto3
import pandas as pd
import psycopg2

s3_client = boto3.client("s3")
bucket_name = "aws-a9097-use1-00-d-s3b-shrd-shr-data01"
IAM_ROLE = "aws-a9097-use1-00-d-rol-shrd-shr-usr01"
check_response = s3_client.list_buckets()
print(check_response)

{'ResponseMetadata': {'RequestId': '9T0G08ZXDQTVYW2D', 'HostId': 'scs/OT8gGEMMlFQL6Jv7YLnud9jDCrI69ucydu66wnIFSezAahZCtQaxQItHXZ5P2jWe3l30nbo=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'scs/OT8gGEMMlFQL6Jv7YLnud9jDCrI69ucydu66wnIFSezAahZCtQaxQItHXZ5P2jWe3l30nbo=', 'x-amz-request-id': '9T0G08ZXDQTVYW2D', 'date': 'Mon, 17 Feb 2025 08:32:48 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'aws-a9097-use1-00-d-s3b-shrd-shr-data01', 'CreationDate': datetime.datetime(2024, 8, 6, 8, 41, 20, tzinfo=tzlocal())}, {'Name': 'cf-templates-1jlshq8udppq6-us-east-1', 'CreationDate': datetime.datetime(2024, 8, 1, 10, 43, 34, tzinfo=tzlocal())}], 'Owner': {'DisplayName': 'aws.a9097', 'ID': '6d9499ce97be2b4c12d262e50a6c0e45aa4f21142385c704f4dffe905958950b'}}


In [10]:
files_list = [
    "GeoHiery.txt",
    "MarketDefinition.txt",
    "Prescriber.txt",
    "PrescriberAddress.txt",
    "ProductMaster.txt",
    "Sales.txt",
    "Zip_To_Terr.txt",
]

# Files to S3
for file in files_list:
    s3_client.upload_file(file, bucket_name, f"raw/{file}")

EXTRACTION

In [16]:
for file in files_list:
    s3_file = f"s3://{bucket_name}/raw/{file}"
    df = pd.read_csv(s3_file, delimiter="|")
    print(df.head())

   GEO_CODE    GEO_NAME GEO_LEVEL PARENT_ID
0  T-111100      Acampo      TERR  D-111000
1  D-111000  District 1      DIST  R-110000
2  R-110000          CA      REGN  N-100000
3  N-100000         USA    NATION         -
4  T-111200       Acton      TERR  D-111000
  MKT_CD          MKT_NM
0  M-001    NEUROSCIENCE
1  M-002        VIROLOGY
2  M-003      IMMUNOLOGY
3  M-004      HEMATOLOGY
4  M-005  CARDIOVASCULAR
  Prescriber_ID FIRST_NAME  LAST_NAME         PREFFERED_NAME  GENDER  EMAIL  \
0      ST_00001   SOONTORN  THRUPKAEW     SOONTORN THRUPKAEW     NaN    NaN   
1      RB_00002     ROBERT      BRYAN           ROBERT BRYAN     NaN    NaN   
2      JC_00003       JOHN     CARTER  JOHN JEFFERSON CARTER     NaN    NaN   
3      TK_00004   THEODORE   KIRKLAND    THEODORE N KIRKLAND     NaN    NaN   
4      TC_00005        TOM   CALDWELL           TOM CALDWELL     NaN    NaN   

   PERSON_SPECIALTY  
0  GastroEnterology  
1         NeuroLogy  
2       Dermatology  
3         NeuroLogy  
4

In [22]:
# REDSHIFT CREDS
REDSHIFT_HOST = "aws-a9097-use1-00-d-red-shrd-shr-red01.ca3wdeu8uqbl.us-east-1.redshift.amazonaws.com"
REDSHIFT_DB = "dev"
REDSHIFT_USER = "group1user"
REDSHIFT_PORT = "5439"
REDSHIFT_PASSWORD = "Group1pass"
conn = psycopg2.connect(
    dbname=REDSHIFT_DB,
    user=REDSHIFT_USER,
    password=REDSHIFT_PASSWORD,
    host=REDSHIFT_HOST,
    port=REDSHIFT_PORT,
)
cursor = conn.cursor()
cursor.execute("SELECT current_date")
print("REDSHIFT CREDS LOADED SUCESSFULLY")

REDSHIFT CREDS LOADED


In [31]:
# Creating respective dataframes to work with for EXTRACT


def extractFromS3(file_name):
    print(file_name)
    obj = s3_client.get_object(Bucket=bucket_name, Key=f"raw/{file}")
    return pd.read_csv(io.bytesIO(obj["Body"].read()))


for file in files_list:
    df[file] = extractFromS3(file)

print(df)

GeoHiery.txt


NameError: name 'io' is not defined

In [60]:
sql1 = f"""CREATE TABLE IF NOT EXISTS bs_grp1_person_profile(
    Prescriber_ID VARCHAR(50),
    FIRST_NAME VARCHAR(100),
    LAST_NAME VARCHAR(100),
    PREFFERED_NAME VARCHAR(100),
    GENDER VARCHAR(10),
    EMAIL VARCHAR(100),
    PERSON_SPECIALTY VARCHAR(100)
);"""


cursor.execute(sql1)

conn.commit()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [63]:
sql1 = """CREATE TABLE IF NOT EXISTS bs_grp1_person_profile(
    Prescriber_ID VARCHAR(50),
    FIRST_NAME VARCHAR(100),
    LAST_NAME VARCHAR(100),
    PREFFERED_NAME VARCHAR(100),
    GENDER VARCHAR(10),
    EMAIL VARCHAR(100),
    PERSON_SPECIALTY VARCHAR(100)
);"""

# Rollback any previous failed transaction
conn.rollback()

# Execute the CREATE TABLE statement
cursor.execute(sql1)
conn.commit()

In [73]:
sql2 = """
COPY bs_grp1_person_profile
FROM 's3://aws-a9097-use1-00-d-s3b-shrd-shr-data01/raw/Prescriber.txt'
DELIMITER '|'
IGNOREHEADER 1
REMOVEQUOTES
EMPTYASNULL
IAM_ROLE 'arn:aws:iam::014498666895:role/service-role/AmazonRedshift-CommandsAccessRole-20240806T150642'
;
"""


# Rollback any previous failed transaction
conn.rollback()

# Execute the CREATE TABLE statement
cursor.execute(sql2)
conn.commit()

In [109]:
sql_create = """CREATE TABLE IF NOT EXISTS bs_grp1_prod_master(
    MARKET_CD VARCHAR(10),
    PRD_PACK_ID VARCHAR(50),
    PRODUCT_NAME VARCHAR(200),
    IMS_STRENG VARCHAR(50),
    TH_LEVEL VARCHAR(50),
    BRAND_ID VARCHAR(50),
    BRAND_NAME VARCHAR(100),
    PROD_MNF VARCHAR(100)
);"""

# Rollback any previous failed transaction
conn.rollback()

# Execute the CREATE TABLE statement
cursor.execute(sql_create)
conn.commit()
print("Successfully created prod_master")

sql_copy = """
COPY bs_grp1_prod_master
FROM 's3://aws-a9097-use1-00-d-s3b-shrd-shr-data01/raw/Prescriber.txt'
DELIMITER '|'
IGNOREHEADER 1
REMOVEQUOTES
EMPTYASNULL
IAM_ROLE 'arn:aws:iam::014498666895:role/service-role/AmazonRedshift-CommandsAccessRole-20240806T150642'
;
"""


# Rollback any previous failed transaction
conn.rollback()

# Execute the CREATE TABLE statement
cursor.execute(sql_copy)
conn.commit()
print("Data loaded in table")

Successfully created prod_master


InternalError_: Load into table 'bs_grp1_prod_master' failed.  Check 'stl_load_errors' system table for details.


In [107]:
# create_person_address
sql_create = """
CREATE TABLE IF NOT EXISTS bs_grp1_person_address (
    PERSON_CODE VARCHAR(50),
    ADDRESS_1 VARCHAR(200),
    ADDRESS_2 VARCHAR(200),
    CITY VARCHAR(100),
    STATE VARCHAR(50),
    ZIP_CODE VARCHAR(10)
);
"""
conn.rollback()
cursor.execute(sql_create)
conn.commit()
print("Created table bs_grp1_person_address successfully!")

# COPY FROM S3
sql_copy = """
COPY bs_grp1_person_address
FROM 's3://aws-a9097-use1-00-d-s3b-shrd-shr-data01/raw/PrescriberAddress.txt'
DELIMITER '|'
IGNOREHEADER 1
EMPTYASNULL
BLANKSASNULL
IAM_ROLE 'arn:aws:iam::014498666895:role/service-role/AmazonRedshift-CommandsAccessRole-20240806T150642';
"""
conn.rollback()
cursor.execute(sql_copy)
conn.commit()
print("Data loaded into bs_grp1_person_address from S3!")

Created table bs_grp1_person_address successfully!
Data loaded into bs_grp1_person_address from S3!


In [102]:
# CREATE TABLE_geo_hierachy
sql_create = """
CREATE TABLE IF NOT EXISTS bs_grp1_geo_hierarchy (
    GEO_CODE   VARCHAR(50),
    GEO_NAME   VARCHAR(100),
    GEO_LEVEL  VARCHAR(20),
    PARENT_ID  VARCHAR(50)
);
"""
conn.rollback()
cursor.execute(sql_create)
conn.commit()
print("Created table bs_grp1_geo_hierarchy successfully!")
# COPY
sql_copy = """
COPY bs_grp1_geo_hierarchy
FROM 's3://aws-a9097-use1-00-d-s3b-shrd-shr-data01/raw/GeoHiery.txt'
DELIMITER '|'
IGNOREHEADER 1
REMOVEQUOTES
EMPTYASNULL
BLANKSASNULL
IAM_ROLE 'arn:aws:iam::014498666895:role/service-role/AmazonRedshift-CommandsAccessRole-20240806T150642';
"""
conn.rollback()
cursor.execute(sql_copy)
conn.commit()
print("Data loaded into bs_grp1_geo_hierarchy from S3!")

Created table bs_grp1_geo_hierarchy successfully!
Data loaded into bs_grp1_geo_hierarchy from S3!


In [101]:
# CREATE TABLE
sql_create3 = """
CREATE TABLE IF NOT EXISTS bs_grp1_zip_terr (
    ZIP_Code      VARCHAR(10),
    Territory_ID  VARCHAR(50)
);
"""
conn.rollback()
cursor.execute(sql_create3)
conn.commit()
print("Created table bs_grp1_zip_terr successfully!")
# COPY
sql_copy3 = """
COPY bs_grp1_zip_terr
FROM 's3://aws-a9097-use1-00-d-s3b-shrd-shr-data01/raw/Zip_To_Terr.txt'
DELIMITER '|'
IGNOREHEADER 1
REMOVEQUOTES
EMPTYASNULL
BLANKSASNULL
IAM_ROLE 'arn:aws:iam::014498666895:role/service-role/AmazonRedshift-CommandsAccessRole-20240806T150642';
"""
conn.rollback()
cursor.execute(sql_copy3)
conn.commit()
print("Data loaded into bs_grp1_zip_terr from S3!")

Created table bs_grp1_zip_terr successfully!
Data loaded into bs_grp1_zip_terr from S3!


In [108]:
# CREATE TABLE
sql_create2 = """
CREATE TABLE IF NOT EXISTS bs_grp1_sales (
    PSBR_ID          VARCHAR(50),
    PROD_ID          VARCHAR(50),
    MONTH_1_NRX_UNIT   VARCHAR(50),
    MONTH_2_NRX_UNIT   VARCHAR(50),
    MONTH_3_NRX_UNIT   VARCHAR(50),
    MONTH_4_NRX_UNIT   VARCHAR(50),
    MONTH_5_NRX_UNIT   VARCHAR(50),
    MONTH_6_NRX_UNIT   VARCHAR(50),
    MONTH_7_NRX_UNIT   VARCHAR(50),
    MONTH_8_NRX_UNIT   VARCHAR(50),
    MONTH_9_NRX_UNIT   VARCHAR(50),
    MONTH_10_NRX_UNIT   VARCHAR(50),
    MONTH_11_NRX_UNIT   VARCHAR(50),
    MONTH_12_NRX_UNIT   VARCHAR(50),
    MONTH_1_TRX_UNIT   VARCHAR(50),
    MONTH_2_TRX_UNIT   VARCHAR(50),
    MONTH_3_TRX_UNIT   VARCHAR(50),
    MONTH_4_TRX_UNIT   VARCHAR(50),
    MONTH_5_TRX_UNIT   VARCHAR(50),
    MONTH_6_TRX_UNIT   VARCHAR(50),
    MONTH_7_TRX_UNIT   VARCHAR(50),
    MONTH_8_TRX_UNIT   VARCHAR(50),
    MONTH_9_TRX_UNIT   VARCHAR(50),
    MONTH_10_TRX_UNIT   VARCHAR(50),
    MONTH_11_TRX_UNIT   VARCHAR(50),
    MONTH_12_TRX_UNIT   VARCHAR(50)
);
"""
conn.rollback()
cursor.execute(sql_create2)
conn.commit()
print("Created table bs_grp1_sales successfully!")
# COPY
sql_copy2 = """
COPY bs_grp1_sales
FROM 's3://aws-a9097-use1-00-d-s3b-shrd-shr-data01/raw/Sales.txt'
DELIMITER '|'
IGNOREHEADER 1
REMOVEQUOTES
EMPTYASNULL
BLANKSASNULL
IAM_ROLE 'arn:aws:iam::014498666895:role/service-role/AmazonRedshift-CommandsAccessRole-20240806T150642';
"""
conn.rollback()
cursor.execute(sql_copy2)
conn.commit()
print("Data loaded into bs_grp1_sales from S3!")

Created table bs_grp1_sales successfully!
Data loaded into bs_grp1_sales from S3!


In [88]:
# CREATE THE TABLE
sql_create = """
CREATE TABLE IF NOT EXISTS bs_grp1_mkt_desc (
    MKT_CD  VARCHAR(10),
    MKT_NM  VARCHAR(200)
);
"""
conn.rollback()
cursor.execute(sql_create)
conn.commit()
print("Created table bs_grp1_mkt_desc successfully!")

# COPY THE TABLE
sql_copy = """
COPY bs_grp1_mkt_desc
FROM 's3://aws-a9097-use1-00-d-s3b-shrd-shr-data01/raw/MarketDefinition.txt'
DELIMITER '|'
IGNOREHEADER 1
REMOVEQUOTES
EMPTYASNULL
BLANKSASNULL
IAM_ROLE 'arn:aws:iam::014498666895:role/service-role/AmazonRedshift-CommandsAccessRole-20240806T150642';
"""
conn.rollback()
cursor.execute(sql_copy)
conn.commit()
print("Data loaded into bs_grp1_mkt_desc from S3!")

Created table bs_grp1_mkt_desc successfully!
Data loaded into bs_grp1_mkt_desc from S3!
